# RRT Lab & Hardware Implementation

## Part 1: RRT Algorithm

 We will use the RRT code to find the trajectory from the starting position to the end goal in the drone cage.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple
from numpy import linalg as LA

%matplotlib notebook

In [ ]:
def conf_free(q: np.ndarray, obstacles: List[Tuple[np.ndarray, float]]) -> bool:
    """
    Check if a configuration is in the free space.
    
    This function checks if the configuration q lies outside of all the obstacles in the connfiguration space.
    
    @param q: An np.ndarray of shape (2,) representing a robot configuration.
    @param obstacles: A list of obstacles. Each obstacle is a tuple of the form (center, radius) representing a circle.
    @return: True if the configuration is in the free space, i.e. it lies outside of all the circles in `obstacles`. 
             Otherwise return False.
    """
     
    for obstacle in obstacles:
        
        # Difference vector
        diff = q - obstacle[0]
        radius = obstacle[1]
        
        # Distance between the configuration and center of the obstacle
        two_norm = LA.norm(diff)
        
        # If the distance between the config and obstacle is less than the radius it means 
        # its colliding, therefore not free (return False)
        if (two_norm**2 <= radius**2):
            return False
    
    return True    
        

def edge_free(edge: Tuple[np.ndarray, np.ndarray], obstacles: List[Tuple[np.ndarray, float]]) -> bool:
    """
    Check if a graph edge is in the free space.
    
    This function checks if a graph edge, i.e. a line segment specified as two end points, lies entirely outside of
    every obstacle in the configuration space.
    
    @param edge: A tuple containing the two segment endpoints.
    @param obstacles: A list of obstacles as described in `config_free`.
    @return: True if the edge is in the free space, i.e. it lies entirely outside of all the circles in `obstacles`. 
             Otherwise return False.
    """
    # For my analysis I am going to take the shortest distance between the edge (line) and the obstacle's center. 
    # Then I will check if it is less than the obstacle's radius, if yes, then the edge intersects the obstacle, thus not free
    # For the shortest distance I will use the formula given in https://en.wikipedia.org/wiki/Distance_from_a_point_to_a_line
    
    p1 = edge[0] # point 1 of the edge
    p2 = edge[1] # point 2 of the edge
    two_norm = LA.norm(p1 - p2)
    
    for obstacle in obstacles:
        # Formula used
        dist = abs((p2[1] - p1[1])*obstacle[0][0] - (p2[0] - p1[0])*obstacle[0][1] + p2[0]*p1[1] - p2[1]*p1[0])
        radius = obstacle[1]
        if ((dist/two_norm) <= radius):
            return False
        else:
            continue
    
    return True
        

In [ ]:
def random_conf(width: float, height: float) -> np.ndarray:
    """
    Sample a random configuration from the configuration space.
    
    This function draws a uniformly random configuration from the configuration space rectangle. The configuration 
    does not necessarily have to reside in the free space.
    
    @param width: The configuration space width.
    @param height: The configuration space height.
    @return: A random configuration uniformily distributed across the configuration space.
    """
    x_coord = width * np.random.rand() - (30*k)
    y_coord = height * np.random.rand() 
    
    conf = np.array([x_coord, y_coord])
    
    return conf

def random_free_conf(width: float, height: float, obstacles: List[Tuple[np.ndarray, float]]) -> np.ndarray:
    """
    Sample a random configuration from the free space.
    
    This function draws a uniformly random configuration from the configuration space
    rectangle that lies in the free space.
    
    @param width: The configuration space width.
    @param height: The configuration space height.
    @param obstacles: The list of configuration space obstacles as defined in `edge_free` and `conf_free`.
    @return: A random configuration uniformily distributed across the configuration space.
    """
    
    # Iterate sampling new configurations until finding one that is free (not conflicting any obstacle)
    while (True): 
        conf = random_conf(width, height)
        if (conf_free(conf, obstacles)): 
            return conf
        


In [ ]:
from numpy import linalg as LA

def nearest_vertex(conf: np.ndarray, vertices: np.ndarray) -> int:
    """
    Finds the nearest vertex to conf in the set of vertices.
    
    This function searches through the set of vertices and finds the one that is closest to 
    conf using the L2 norm (Euclidean distance).
    
    @param conf: The configuration we are trying to find the closest vertex to.
    @param vertices: The set of vertices represented as an np.array with shape (n, 2). Each row represents
                     a vertex.
    @return: The index (i.e. row of `vertices`) of the vertex that is closest to `conf`.
    """
    no_rows = vertices.shape[0]
    min_dist = 100000
    index = 0
    
    # Go through every element in the vertices array and check if they are the closest to the conf vertex
    for i in range(no_rows):
        euc_dist = LA.norm(conf - vertices[i])

        if (euc_dist < min_dist):
            min_dist = euc_dist
            index = i
            
    return index
    
    

def extend(origin: np.ndarray, target: np.ndarray, step_size: float=0.2) -> np.ndarray:
    """
    Extends the RRT at most a fixed distance toward the target configuration.
    
    Given a configuration in the RRT graph `origin`, this function returns a new configuration that takes a
    step of at most `step_size` towards the `target` configuration. That is, if the L2 distance between `origin`
    and `target` is less than `step_size`, return `target`. Otherwise, return the configuration on the line
    segment between `origin` and `target` that is `step_size` distance away from `origin`.
    
    @param origin: A vertex in the RRT graph to be extended.
    @param target: The vertex that is being extended towards.
    @param step_size: The maximum allowed distance the returned vertex can be from `origin`.
    
    @return: A new configuration that is as close to `target` as possible without being more than
            `step_size` away from `origin`.
    """

    # If L2 distance is less than step size return target
    if LA.norm(origin - target) < step_size:
        return target
    
    # Otherwise return the configuration on the line segment between origin and target of step size
    else:
        diff = target - origin
        two_norm = LA.norm(diff)
        unit_vector = diff / two_norm # Unit vector of the line segment between origin and target
        qs = unit_vector * step_size
        return (qs + origin)
        

In [ ]:
def rrt(origin: np.ndarray, width: float, height: float, obstacles: List[Tuple[np.ndarray, float]],
        trials: int=1000, step_size: float=0.13) -> (np.ndarray, np.ndarray):
    """
    Explore a workspace using the RRT algorithm.
    
    This function builds an RRT using `trials` samples from the free space.
    
    @param origin: The starting configuration of the robot.
    @param width: The width of the configuration space.
    @param height: The height of the configuration space.
    @param obstacles: A list of circular obstacles.
    @param trials: The number of configurations to sample from the free space.
    @param step_size: The step_size to pass to `extend`.
    
    @return: A tuple (`vertices`, `parents`), where `vertices` is an (n, 2) `np.ndarray` where each row is a configuration vertex
             and `parents` is an array identifying the parent, i.e. `parents[i]` is the parent of the vertex in
             the `i`th row of `vertices.
    """
    num_verts = 1
    
    vertices = np.zeros((trials + 1, len(origin)))
    vertices[0, :] = origin
    
    parents = np.zeros(trials + 1, dtype=int)
    parents[0] = -1
    
    for trial in range(trials):
        
        # Randomly sample a configuration
        qrand = random_conf(width, height)
        
        # Find vertex that is closest to qrand
        near_row_index = nearest_vertex(qrand, vertices)
        qnear = vertices[near_row_index, :]
    
        # Extend qnear towards qrand
        qs = extend(qnear, qrand)
        
        edge = (qnear, qs)
        # Check if qs is in collision
        if conf_free(qs, obstacles) and edge_free(edge, obstacles): 
            # Add qs to V
            vertices[num_verts, :] = qs
            # Record the parent of the new configuration
            parents[num_verts] = near_row_index
            num_verts += 1
            
        else :
            continue
            
    
    return vertices[:num_verts, :], parents[:num_verts]

def backtrack(index: int, parents: np.ndarray) -> List[int]:
    """
    Find the sequence of nodes from the origin of the graph to an index.
    
    This function returns a List of vertex indices going from the origin vertex to the vertex `index`.
    
    @param index: The vertex to find the path through the tree to.
    @param parents: The array of vertex parents as specified in the `rrt` function.
    
    @return: The list of vertex indicies such that specifies a path through the graph to `index`.
    """
    
    path_list = []
    i = parents[index]
    # While we are not at the origin (where its -1) keep finding parents and added them to the path list
    while i != -1:
        path_list.append(i)
        i = parents[i]
    
    return path_list

## Part 2: Defining the Configuration Space

The drone cage has been set up such that a series of PVC pipes are suspended from the ceiling.

In [ ]:
k = 0.0254 #inches to meters
# Width and height of the rectangular domain: 
width = 96.0*k  # total width of the cage
height = 150.0*k # total height of the cage

# Obstacles are represented as tuples, where the first element is an np.ndarray containing the center
# and the second element is the radius of the obstacle. For example (np.array([3, 4]), 5).
# This variable is a list of such tuples.
obstacles = []

# Measurements for the obstacles in meters
obstacles.append((np.array([-10*k,44*k]), (1+4)*k))
obstacles.append((np.array([33*k, 33*k]), (2.25+4)*k))
obstacles.append((np.array([4*k,70*k]), (.75+4)*k))
obstacles.append((np.array([23*k,68*k]), (1.75+4)*k))
obstacles.append((np.array([44*k,82*k]), (2.25+4)*k))


# The goal is represented in the same way as an obstacle.
# We tried using the actual x direction (120) for the drone, however, after 10+ trials it was 
# falling short each time. So we decided the scale it upwards to account for the uncertainities 
# in the sensor measurements.
goal = (np.array([45.7*k, 140*k]), (4)*k)

# The starting position of the robot.
origin = np.array([0, 0])

# Run the RRT to find the trajectory in this space: 
vertices, parents = rrt(origin, width, height, obstacles)

# Check if path was found: 
index = nearest_vertex(goal[0], vertices)
if np.linalg.norm(vertices[index, :] - goal[0]) < goal[1]:
    print('Path found!')
    path_verts = backtrack(index, parents)
else:
    print('No path found!')
    path_verts = []

In [ ]:
## Sample code to plot the trajectory: 
## Plot is flipped to match positive values of drone 
## origin is (0,0) to match drone
## scale in meters

fig, ax = plt.subplots()

ax.set_xlim([-30 *k, width -(k*30)])
ax.set_ylim([0, height])
ax.set_aspect('equal')

for i in range(len(parents)):
    if parents[i] < 0:
        continue
    plt.plot([vertices[i, 0], vertices[parents[i], 0]], 
             [vertices[i, 1], vertices[parents[i], 1]], c='k')

for i in path_verts:
    if parents[i] < 0:
        continue
    plt.plot([vertices[i, 0], vertices[parents[i], 0]], 
             [vertices[i, 1], vertices[parents[i], 1]], c='r')    

for o in obstacles:
    ax.add_artist(plt.Circle(tuple(o[0]), o[1]))
    
ax.add_artist(plt.Circle(tuple(goal[0]), goal[1], ec=(0.004, 0.596, 0.105), fc=(1, 1, 1)))

plt.scatter([2.5], [3.5], zorder=3, c=np.array([[0.004, 0.596, 0.105]]), s=3)
plt.scatter(vertices[path_verts, 0], vertices[path_verts, 1], c=np.array([[1, 0, 0]]), s=3, zorder=2)
plt.scatter(vertices[1:, 0], vertices[1:, 1], c=np.array([[0, 0, 0]]), s=3)

## Part 3: Hardware Implementation.

Now that we have a trajectory, we can begin testing with the drone. 

In [ ]:
group_number = 10

In [ ]:
def seg_to_setpoints(start_conf: np.ndarray, end_conf: np.ndarray) -> np.ndarray:
    # This function takes in the RRT trajectory and outputs a sequence of setpoints for the drone to follow. 
    
    dist = np.linalg.norm(start_conf - end_conf)
    num_samples = int(100 * dist)
    
    return end_conf.reshape((1, 2))

traj = origin.reshape(1, 2)
for i in range(len(path_verts) - 1):
    traj = np.concatenate((traj, seg_to_setpoints(vertices[path_verts[i]], vertices[path_verts[i + 1]])))

Here, we provide the code which actually tells the drone how to follow a sequence of setpoints:

In [ ]:
# Code adapted from: https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/autonomousSequence.py

import time
# CrazyFlie imports: 
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger

## Some helper functions: 
## -----------------------------------------------------------------------------------------

# Determine initial position: 
def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001
    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            print("{} {} {}".
                format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break

# Initialize controller: 
def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    
    wait_for_position_estimator(cf)
    time.sleep(0.1)    
    return

# Ascend and hover: 
def ascend_and_hover(cf):
    # Ascend:
    for y in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 50)
        time.sleep(0.1)
    # Hover at 0.5 meters:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.5)
        time.sleep(0.1)
    return

# Follow the setpoint sequence trajectory:
def run_sequence(scf, sequence, setpoint_delay):
    cf = scf.cf
    for position in sequence:
        print(f'Setting position {(position[0], (position[1]))}')
        for i in range(setpoint_delay):
            cf.commander.send_position_setpoint(position[0],
                                                (position[1]),
                                                0.5,
                                                0.0)
            time.sleep(0.1)
    
# Hover, descend, and stop all motion:
def hover_and_descend(cf):
    # Hover at 0.5 meters:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.5)
        time.sleep(0.1)
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 25)
        time.sleep(0.1)
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
    return
## -----------------------------------------------------------------------------------------

def run_setpoint_trajectory(group_number, sequence):  
    # This is the main function to enable the drone to follow the trajectory. 
    
    # User inputs:
    #
    # - group_number: (int) the number corresponding to the drone radio settings. 
    #
    # - sequence: a series of point locations (float) defined as a numpy array, with each row in the following format: 
    #     [x(meters), y(meters)]
    #   Note: the input should be given in drone coordinates (where positive x is forward, and positive y is to the left).
    # Example: 
    # sequence = [
    #     [[ 0.          0.        ]
    #      [0.18134891  0.08433607]]
    # 

    # Outputs: 
    # None. 
    
    setpoint_delay = 3 # (int) Time to give the controller to reach next setpoint; in multiples of 0.1 s
    
    # Set the URI the Crazyflie will connect to
    uri = f'radio://0/{group_number}/2M'
    
    # Initialize all the CrazyFlie drivers:
    cflib.crtp.init_drivers(enable_debug_driver=False)

    # Sync to the CrazyFlie: 
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf

        # Initialize and ascend:
        set_PID_controller(cf)
        ascend_and_hover(cf)
        # Run the waypoint sequence:
        run_sequence(scf, sequence, setpoint_delay)
        # Descend and stop all motion: 
        hover_and_descend(cf)
            
    print('Done!') 
    return

In [ ]:
traj = np.zeros((len(path_verts),2))
z = 0
# change x and y to match drone orientation 
for x in path_verts[::-1]:
    traj[z] = (np.array([vertices[x][1],vertices[x][0]]))
    z = z + 1

traj

In [ ]:
# Run the setpoint sequence on the drone: 
run_setpoint_trajectory(group_number, traj)